In [100]:
from scipy import sparse
import numpy as np
import pandas as pd
from numpy.random import choice
import sklearn
from sklearn.preprocessing import normalize

In [ ]:
#sampling

In [101]:
sparse_adj_matrix = sparse.load_npz("sparse_adj_matrix.npz")

In [102]:
sparse_adj_matrix.shape

(10047, 10047)

In [106]:
def myfunction(a_series):
    all_indices = np.array([i+1 for i in range(10047)])
    all_weights = a_series.to_numpy()
    all_weights /= sum(all_weights)
    neighbors = []
    #sample 10 neighbors (or however many possible)
    neighbors = np.random.choice(all_indices, min(10,len(np.nonzero(all_weights)[0])), p=all_weights, replace=False)
    return neighbors

#k = 1 for now
def sampling(sparse_adj_matrix, beta):
    k = 1
    w = pd.DataFrame(sparse_adj_matrix.power(beta).multiply(k).toarray())
    w["u"] = w.apply(lambda a_series: myfunction(a_series), axis=1)
    graph = w[["u"]]
    return graph

In [107]:
#beta = 3 for now
graph = sampling(sparse_adj_matrix, 3)
graph["v"] = np.arange(sparse_adj_matrix.shape[0])+1
graph2 = graph.explode("u")
graph3 = graph2[["u","v"]]

In [108]:
graph3.reset_index(drop=True, inplace=True)

In [109]:
graph3.head(10)

,u,v
0,7776,1
1,4643,1
2,8775,1
3,9601,1
4,9737,1
5,10,1
6,9383,1
7,7811,1
8,5452,1
9,7382,1


In [67]:
graph3["u,v"] = graph3["u"].astype(str)+','+graph3["v"].astype(str)

In [68]:
graph3.head()

,u,v,"u,v"
0,9866,1,"9866,1"
1,7494,1,"7494,1"
2,8956,1,"8956,1"
3,4643,1,"4643,1"
4,5452,1,"5452,1"


In [69]:
graph3["s(u,v)"] = graph3["u,v"].apply(lambda x: sparse_adj_matrix[int(x.split(',')[0])-1,int(x.split(',')[1])-1])

In [70]:
graph3.drop(columns=["u,v"], inplace=True)

In [71]:
graph3.head()

,u,v,"s(u,v)"
0,9866,1,0.321751
1,7494,1,0.117884
2,8956,1,0.308931
3,4643,1,0.244979
4,5452,1,0.239583


In [61]:
#aggregation
#goes up to the CONCAT() part of the algorithm for the first layer

In [72]:
item_vectors = np.load('item_vectors.npy')

In [73]:
graph3["item vector of u"] = graph3["u"].apply(lambda x: item_vectors[x-1].tolist())

In [74]:
graph3.head()

,u,v,"s(u,v)",item vector of u
0,9866,1,0.321751,"[-0.00041886905091814697, 0.002327881054952740..."
1,7494,1,0.117884,"[-0.0014167504850775003, -0.000661931582726538..."
2,8956,1,0.308931,"[-0.0019281146815046668, -0.002377113793045282..."
3,4643,1,0.244979,"[0.0007616159855388105, -0.0012145633809268475..."
4,5452,1,0.239583,"[0.002006744733080268, 0.002265156479552388, 5..."


In [75]:
graph3["item vector of u normalized"] = graph3["item vector of u"].apply(lambda x: np.array(x) / np.sqrt(np.sum(np.array(x)**2)))

In [76]:
graph3.head()

,u,v,"s(u,v)",item vector of u,item vector of u normalized
0,9866,1,0.321751,"[-0.00041886905091814697, 0.002327881054952740...","[-0.019957349291730386, 0.11091374552848568, -..."
1,7494,1,0.117884,"[-0.0014167504850775003, -0.000661931582726538...","[-0.0680055548405659, -0.03177343154207547, -0..."
2,8956,1,0.308931,"[-0.0019281146815046668, -0.002377113793045282...","[-0.1024934531319508, -0.12636105283253646, 0...."
3,4643,1,0.244979,"[0.0007616159855388105, -0.0012145633809268475...","[0.03888659880051973, -0.06201319274895704, -0..."
4,5452,1,0.239583,"[0.002006744733080268, 0.002265156479552388, 5...","[0.09638903224587536, 0.10880120293844839, 0.0..."


In [77]:
#gamma = 3 for now
graph3["(s(u,v)**gamma)*(item vector of u normalized)"] = (graph3["s(u,v)"]**3)*(graph3["item vector of u normalized"])

In [78]:
graph3.head()

,u,v,"s(u,v)",item vector of u,item vector of u normalized,"(s(u,v)**gamma)*(item vector of u normalized)"
0,9866,1,0.321751,"[-0.00041886905091814697, 0.002327881054952740...","[-0.019957349291730386, 0.11091374552848568, -...","[-0.0006647537103691717, 0.0036943946209106134..."
1,7494,1,0.117884,"[-0.0014167504850775003, -0.000661931582726538...","[-0.0680055548405659, -0.03177343154207547, -0...","[-0.0001114075101401435, -5.205161409255069e-0..."
2,8956,1,0.308931,"[-0.0019281146815046668, -0.002377113793045282...","[-0.1024934531319508, -0.12636105283253646, 0....","[-0.0030218960150685706, -0.003725603444377219..."
3,4643,1,0.244979,"[0.0007616159855388105, -0.0012145633809268475...","[0.03888659880051973, -0.06201319274895704, -0...","[0.0005717217842516247, -0.0009117355155550343..."
4,5452,1,0.239583,"[0.002006744733080268, 0.002265156479552388, 5...","[0.09638903224587536, 0.10880120293844839, 0.0...","[0.0013255490120845346, 0.0014962420900832387,..."


In [80]:
aggregated = pd.DataFrame(graph3.groupby('v')['(s(u,v)**gamma)*(item vector of u normalized)'].apply(np.mean))

In [81]:
aggregated.head()

,"(s(u,v)**gamma)*(item vector of u normalized)"
v,
1,"[0.00043145295131681376, -0.001227778805448701..."
2,"[-0.008280149388059387, 0.014760684170112306, ..."
3,"[-0.0002361671881920575, 0.0018971397607103382..."
4,"[-0.02756260864145301, -0.004371294304897443, ..."
5,"[5.457826251767776e-07, -6.001193566735159e-06..."


In [82]:
aggregated.reset_index(inplace=True)

In [83]:
aggregated.head()

,v,"(s(u,v)**gamma)*(item vector of u normalized)"
0,1,"[0.00043145295131681376, -0.001227778805448701..."
1,2,"[-0.008280149388059387, 0.014760684170112306, ..."
2,3,"[-0.0002361671881920575, 0.0018971397607103382..."
3,4,"[-0.02756260864145301, -0.004371294304897443, ..."
4,5,"[5.457826251767776e-07, -6.001193566735159e-06..."


In [84]:
aggregated.rename(columns={"(s(u,v)**gamma)*(item vector of u normalized)": "h_k_Nv"}, inplace=True)

In [85]:
aggregated["h_k_Nv normalized"] = aggregated["h_k_Nv"].apply(lambda x: np.array(x) / np.sqrt(np.sum(np.array(x)**2)))

In [86]:
aggregated.head()

,v,h_k_Nv,h_k_Nv normalized
0,1,"[0.00043145295131681376, -0.001227778805448701...","[0.03412164397916872, -0.09709941989463228, -0..."
1,2,"[-0.008280149388059387, 0.014760684170112306, ...","[-0.04987246520650074, 0.08890560703646962, 0...."
2,3,"[-0.0002361671881920575, 0.0018971397607103382...","[-0.014174424503280138, 0.11386367647520647, -..."
3,4,"[-0.02756260864145301, -0.004371294304897443, ...","[-0.12043331252392313, -0.01910013163137255, 0..."
4,5,"[5.457826251767776e-07, -6.001193566735159e-06...","[0.0026428237075869217, -0.02905936521311713, ..."


In [87]:
h_0 = sklearn.preprocessing.normalize(item_vectors, norm="l2")

In [88]:
aggregated["h_0_v"] = aggregated["v"].apply(lambda x: h_0[x-1])

In [89]:
aggregated.head()

,v,h_k_Nv,h_k_Nv normalized,h_0_v
0,1,"[0.00043145295131681376, -0.001227778805448701...","[0.03412164397916872, -0.09709941989463228, -0...","[-0.025009742, -0.061546545, -0.021576503, 0.0..."
1,2,"[-0.008280149388059387, 0.014760684170112306, ...","[-0.04987246520650074, 0.08890560703646962, 0....","[0.06487809, 0.018574934, -0.06885929, -0.1202..."
2,3,"[-0.0002361671881920575, 0.0018971397607103382...","[-0.014174424503280138, 0.11386367647520647, -...","[-0.043342058, -0.039694242, -0.08958718, -0.1..."
3,4,"[-0.02756260864145301, -0.004371294304897443, ...","[-0.12043331252392313, -0.01910013163137255, 0...","[0.01583572, -0.11006086, 0.0691011, -0.048232..."
4,5,"[5.457826251767776e-07, -6.001193566735159e-06...","[0.0026428237075869217, -0.02905936521311713, ...","[0.02773153, -0.047105808, 0.042153057, -0.057..."


In [90]:
a = np.array([3, 4])
b = np.array([5, 6])
np.concatenate([a, b])

array([3, 4, 5, 6])

In [91]:
h_0_v = aggregated["h_0_v"].tolist()
h_k_Nv_normalized = aggregated["h_k_Nv normalized"].tolist()

concatenated = []
for i in range(len(h_0_v)):
    concatenated.append(np.concatenate([h_0_v[i], h_k_Nv_normalized[i]])) 

aggregated["concatenated"] = concatenated
aggregated.head()

,v,h_k_Nv,h_k_Nv normalized,h_0_v,concatenated
0,1,"[0.00043145295131681376, -0.001227778805448701...","[0.03412164397916872, -0.09709941989463228, -0...","[-0.025009742, -0.061546545, -0.021576503, 0.0...","[-0.025009742006659508, -0.06154654547572136, ..."
1,2,"[-0.008280149388059387, 0.014760684170112306, ...","[-0.04987246520650074, 0.08890560703646962, 0....","[0.06487809, 0.018574934, -0.06885929, -0.1202...","[0.06487809121608734, 0.01857493445277214, -0...."
2,3,"[-0.0002361671881920575, 0.0018971397607103382...","[-0.014174424503280138, 0.11386367647520647, -...","[-0.043342058, -0.039694242, -0.08958718, -0.1...","[-0.04334205761551857, -0.03969424217939377, -..."
3,4,"[-0.02756260864145301, -0.004371294304897443, ...","[-0.12043331252392313, -0.01910013163137255, 0...","[0.01583572, -0.11006086, 0.0691011, -0.048232...","[0.01583571918308735, -0.11006086319684982, 0...."
4,5,"[5.457826251767776e-07, -6.001193566735159e-06...","[0.0026428237075869217, -0.02905936521311713, ...","[0.02773153, -0.047105808, 0.042153057, -0.057...","[0.027731530368328094, -0.047105807811021805, ..."
